# This file is for trial to see various conflicts, DO NOT USE THIS

In [1]:
print("hello hi")

hello hi


In [2]:
pip install transformers torch datasets nltk pdfplumber EbookLib


     |████████████████████████████████| 59 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 5.2 MB/s eta 0:00:01
  Created wheel for EbookLib: filename=EbookLib-0.18-py3-none-any.whl size=38794 sha256=3b12624d272ece929cf9a66bb0cee4d967b95cada58ece73a725bb351669446d
  Stored in directory: /Users/sparshpatel/Library/Caches/pip/wheels/47/3a/ec/289c2f96d54695a17d260684be304d20a8d0bf50b08b75862e
Successfully built EbookLib
Note: you may need to restart the kernel to use updated packages.


In [21]:
#Extracting the text from the book

from ebooklib import epub
from bs4 import BeautifulSoup

def extract_text_from_epub(epub_path):
    # Open the EPUB file
    book = epub.read_epub(epub_path)
    
    # Extract text from the book's HTML content
    text = ""
    for item in book.get_items():
        if item.media_type == "application/xhtml+xml":  # Correct way to identify text content
            soup = BeautifulSoup(item.content, "html.parser")
            text += soup.get_text() + "\n\n"  # Add spacing between sections
    
    return text

# Example usage
epub_path = "/Users/sparshpatel/Documents/Codes/codes/pythonml/slm/Percy Jackson and the Olympians 5 - The Last Olympian by Rick Riordan (z-lib.org).epub"  
book_text = extract_text_from_epub(epub_path)

# Print first 500 characters to check
print(book_text[500:])


/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


.............................................................................................................................. 21 
Chapter Five ..................................................................................................................................... 26 
Chapter Six ....................................................................................................................................... 30 
Chapter Seven .................................................................................................................................. 37 
Chapter Eight .................................................................................................................................... 44 
Chapter Nine ..................................................................................................................................... 49 
Chapter Ten .......................................................................................................

In [22]:
#Tokenising the data

import re

def clean_text(text):
    # Remove extra spaces and newlines
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces/newlines with a single space
    text = text.strip()
    
    # Remove unwanted characters (optional)
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters (if needed)

    return text

# Apply cleaning
cleaned_text = clean_text(book_text)

# Print a small portion to check
print(cleaned_text[10000:15000])


r. "Time?" I asked. He nodded grimly. A clump formed in my throat. I'd known this was coming. We'd been planning for it for weeks, but I'd half hoped it would never happen. Rachel looked up at Beckendorf. "Hi." "Oh, hey. I'm Beckendorf. You must be Rachel. Percy's told me . . . uh, I mean he mentioned you." Rachel raised an eyebrow. "Really? Good." She glanced at Blackjack, who was clopping his hooves against the hood of the Prius. "So I guess you guys have to go save the world now." "Pretty much," Beckendorf agreed. I looked at Rachel helplessly. "Would you tell my mom" "I'll tell her. I'm sure she's used to it. And I'll explain to Paul about the hood." I nodded my thanks. I figured this might be the last time Paul loaned me his car. "Good luck." Rachel kissed me before I could even react. "Now, get going, half-blood. Go kill some monsters for me." My last view of her was sitting in the shotgun seat of the Prius, her arms crossed, watching Rick Riordan Percy Jackson and the Olympians 

In [23]:
#Classifying the tokens into chunks

import nltk
nltk.download('punkt')
import ssl
import nltk
import certifi
import nltk
nltk.download('punkt_tab')

ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())



from nltk.tokenize import sent_tokenize

def split_into_chunks(text, max_chunk_size=512):
    sentences = sent_tokenize(text)  # Split into sentences
    chunks = []
    chunk = ""

    for sentence in sentences:
        if len(chunk) + len(sentence) <= max_chunk_size:
            chunk += sentence + " "
        else:
            chunks.append(chunk.strip())
            chunk = sentence + " "
    
    if chunk:  # Add the last chunk
        chunks.append(chunk.strip())

    return chunks

# Apply chunking
chunks = split_into_chunks(cleaned_text)

# Print the first few chunks
for i, chunk in enumerate(chunks[7:15]):
    print(f"\nChunk {i+1}:\n{chunk}\n{'-'*50}")



Chunk 1:
She'd even been there last summer when the chopped-up Lord Kronos rose out of his coffin in a terrible new form, and she'd earned my permanent respect by nailing him in the eye with a blue plastic hairbrush. She put her hand on my arm. "Just think about it, okay? We don't leave for a couple of days. My dad . . ." Her voice faltered. "Is he giving you a hard time?" I asked. Rachel shook her head in disgust. "He's trying to be nice to me, which is almost worse.
--------------------------------------------------

Chunk 2:
He wants me to go to Clarion Ladies Academy m the fall." "The school where your mom went?" "It's a stupid finishing school for society girls, all the way in New Hampshire. Can you see me in finishing school?" I admitted the idea sounded pretty dumb. Rachel was into urban art projects and feeding the homeless and going to protest rallies to "Save the Endangered Yellow-bellied Sapsucker" and stuff like that. I'd never even seen her wear a dress. It was hard to im

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sparshpatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/sparshpatel/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [24]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Example sentences
sentences = ["This is a sentence.", "Here is another one."]

# Tokenize sentences
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)

# Generate embeddings
with torch.no_grad():
    outputs = model(**inputs)

# The embeddings are in the last_hidden_state
embeddings = outputs.last_hidden_state.mean(dim=1)  # Pooling the output (mean of all tokens)

print(embeddings)


/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


tensor([[ 0.0884, -0.1475,  0.1752,  ..., -0.0535, -0.1021,  0.5540],
        [ 0.1275, -0.2232,  0.1702,  ..., -0.0162,  0.0776,  0.1723]])


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load model and tokenizer (Assuming you are using the same BERT model)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

def get_embedding(text):
    # Tokenize and get embedding
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Return the mean of the last hidden state (embedding for the sentence)
    return outputs.last_hidden_state.mean(dim=1)

# Assume `chunks` contains the chunks of text from the book
# Example question
question = "What happens at the end of the book?"

# Get the question embedding
question_embedding = get_embedding(question)

# Calculate cosine similarities with the chunks
chunk_embeddings = [get_embedding(chunk) for chunk in chunks]  # Precompute for chunks

similarities = [cosine_similarity(question_embedding, chunk_embedding)[0][0] for chunk_embedding in chunk_embeddings]

# Find the most similar chunk
most_similar_chunk_index = similarities.index(max(similarities))
most_similar_chunk = chunks[most_similar_chunk_index]

print("Most relevant chunk:", most_similar_chunk)


ImportError: tokenizers>=0.20,<0.21 is required for a normal functioning of this module, but found tokenizers==0.21.0.
Try: `pip install transformers -U` or `pip install -e '.[dev]'` if you're working with git main

In [1]:
import sklearn
print(sklearn.__version__)


1.0.2


In [ ]:
import sys
print(sys.executable)


/usr/local/bin/python3


In [ ]:
import sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load model
model = SentenceTransformer('bert-base-nli-mean-tokens')

def get_embedding(text):
    return model.encode(text)

# Calculate the embeddings
question_embedding = get_embedding(question)
chunk_embeddings = [get_embedding(chunk) for chunk in chunks]

# Calculate cosine similarities
similarities = [cosine_similarity([question_embedding], [chunk_embedding])[0][0] for chunk_embedding in chunk_embeddings]

# Find most relevant chunk
most_similar_chunk_index = similarities.index(max(similarities))
most_similar_chunk = chunks[most_similar_chunk_index]
print("Most relevant chunk:", most_similar_chunk)


In [1]:
print("hello hi")

hello hi


In [10]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup

def extract_text_from_epub(epub_path):
    book = epub.read_epub(epub_path)
    text = ""
    
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.content, 'html.parser')
            text += soup.get_text() + "\n\n"
    
    return text

# Example Usage
epub_path = "/Users/sparshpatel/Documents/Codes/codes/pythonml/slm/Percy Jackson and the Olympians 5 - The Last Olympian by Rick Riordan (z-lib.org).epub"  # Replace with your actual file
book_text = extract_text_from_epub(epub_path)
print(book_text[:1000])  # Print a sample


/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):





index








Contents 
Chapter One ....................................................................................................................................... 1 
Chapter Two ........................................................................................................................................ 8 
Chapter Three ................................................................................................................................... 14 
Chapter Four ..................................................................................................................................... 21 
Chapter Five ..................................................................................................................................... 26 
Chapter Six ....................................................................................................................................... 30 
Chapter Seven ....................................................

In [11]:
import nltk
from nltk.tokenize import sent_tokenize

# Download necessary NLTK data
nltk.download('punkt')

def split_into_chunks(text, max_chunk_size=512):
    sentences = sent_tokenize(text)  # Split into sentences
    chunks = []
    chunk = ""

    for sentence in sentences:
        if len(chunk) + len(sentence) <= max_chunk_size:
            chunk += sentence + " "
        else:
            chunks.append(chunk.strip())
            chunk = sentence + " "

    if chunk:
        chunks.append(chunk.strip())

    return chunks

# Apply chunking
chunks = split_into_chunks(book_text)
print(f"Total chunks: {len(chunks)}")
print(chunks[:3])  # Print first 3 chunks for verification


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sparshpatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Total chunks: 1065
['', 'index\n\n\n\n\n\n\n\n\nContents \nChapter One ....................................................................................................................................... 1 \nChapter Two ........................................................................................................................................ 8 \nChapter Three ................................................................................................................................... 14 \nChapter Four ..................................................................................................................................... 21 \nChapter Five ..................................................................................................................................... 26 \nChapter Six ....................................................................................................................................... 30 \nChapter Seven ..............

In [13]:
import faiss
import numpy as np

index = faiss.IndexFlatL2(128)  # Create a test FAISS index
print("FAISS is working!")


FAISS is working!


model = SentenceTransformer("all-MiniLM-L6-v2")  # Use a lightweight model
sample_embedding = model.encode(["This is a test sentence"])
print(sample_embedding.shape)  # Should be (1, 384) or similar


In [19]:
from sentence_transformers import SentenceTransformer

try:
    model = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L6-v2")
    sample_embedding = model.encode(["This is a test sentence"])
    print(sample_embedding.shape)  # Should be (1, 384)
except Exception as e:
    print("Error loading model:", e)


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/huggingface_hub/__init__.py)

In [18]:
from sentence_transformers import SentenceTransformer

# Using an alternative stable model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

print("New Model Loaded Successfully!")


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/huggingface_hub/__init__.py)

In [14]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the same model we used before
model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

# Convert text chunks into embeddings
chunk_embeddings = model.encode(chunks, convert_to_tensor=True)

print("Chunks successfully embedded!")


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/huggingface_hub/__init__.py)

In [29]:
from torch.nn.functional import cosine_similarity
import torch

def find_best_chunk(question):
    # Encode the question
    question_embedding = model.encode(question, convert_to_tensor=True)
    
    # Compute similarity with all chunks
    similarities = cosine_similarity(question_embedding, chunk_embeddings)
    
    # Get the index of the most relevant chunk
    best_chunk_idx = torch.argmax(similarities).item()
    
    return chunks[best_chunk_idx]  # Return the most relevant chunk


In [30]:
from transformers import pipeline

# Load text generation model
qa_model = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2")

def answer_question(question):
    # Get the most relevant chunk
    context = find_best_chunk(question)
    
    # Format prompt for better responses
    prompt = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
    
    # Generate answer
    response = qa_model(prompt, max_length=150, do_sample=True)
    
    return response[0]["generated_text"]


OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2 and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.

In [34]:
pip install transformers accelerate torch


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached huggingface_hub-0.28.1-py3-none-any.whl.metadata (13 kB)
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
Using cached huggingface_hub-0.28.1-py3-none-any.whl (464 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 5.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load an open-source model
model_name = "mosaicml/mpt-1b-redpajama-200b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

# Define text generation pipeline
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=150)


In [7]:
from transformers import pipeline

# Load the Question Answering pipeline
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")



In [ ]:
def answer_question(question, chunks):
    best_answer = ""
    best_score = 0

    for chunk in chunks:
        try:
            result = qa_pipeline(question=question, context=chunk)
            if result["score"] > best_score:  # Select the most confident answer
                best_score = result["score"]
                best_answer = result["answer"]
        except Exception as e:
            print(f"Error processing chunk: {e}")

    return best_answer if best_answer else "No relevant answer found."

# Example usage:
question = "What is Percy Jackson's prophecy?"
answer = answer_question(question, chunks)  # 'chunks' should be your preprocessed text chunks
print("Answer:", answer)


Error processing chunk: `context` cannot be empty
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error processing chunk: Numpy is not available
Error proc